## <font color=#B2D732> <span style="background-color: #4424D6"> Imports

In [73]:
%cd /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/
import sys,json
import glob, os
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/")

sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0/scripts") 
from spinalcordtoolbox.utils.sys import run_proc

from seed_to_voxels import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
# to do: C5toC6 => D,V,R,VI

## <font color=#B2D732> <span style="background-color: #4424D6"> A/ Initialization

 ### <font color=#4424D6> I. Configuration & parameters </font >

In [75]:
# Load config file ------------------------------------------------------------
with open('../config/config_CL.json') as config_file: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_file) # load config file should be open first and the path inside modified
    config['list_subjects']=config["list_subjects_2"]
signal='raw'
    
seed_names=['spinalcord_ica-C6-V']
target_name=['brain_mask'] # define structure target


### <font color=#4424D6> II. Select files </font>

In [86]:
moco_mean=[];data_target=[];ts_target_txt=[];ts_seed_txt ={};data_seed=[];mask_seed={};mask_target=[];mask_target_main=[];mask_seed_main={}; mask_target_inT1w=[]
warp_PAM50_T1w=[];warp_T1w_func=[];warp_from_PAM50_to_T1w_to_func=[];warp_from_func_to_T1w_to_PAM50=[];warp_brain_iy=[]; warp_brain_func=[];moco_brain_mean=[]
#>>> Select data for extraction and target files -------------------------------------
for subject_name in config['list_subjects']:
    # select target files:
    mask_target_main.append(glob.glob(config["main_dir"] + config["masks"][target_name[0].split('_')[0]])[0].split('.gz')[0])
        
    moco_mean.append(glob.glob('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-'+subject_name+'/func/3_MOCO/spinalcord/*mean.nii.gz')[0])
    moco_brain_mean.append(glob.glob('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-'+subject_name+'/func/3_MOCO/brain/*mean.nii.gz')[0])
   
    warp_brain_iy.append(glob.glob('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-'+subject_name+'/anat/brain/segmentation/mri/iy*')[0])
    warp_brain_func.append(glob.glob('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-' + subject_name  + '/func/5_Coregistration/brain/*reg.mat')[0])
                
        
    warp_PAM50_T1w.append(glob.glob('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-'+subject_name+'/anat/T1w_sc_coreg_new_CL/sub-*from-PAM50_to-T1w_mode-image_xfm.nii.gz')[0])
    warp_T1w_func.append(glob.glob('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-'+subject_name+'/func//5_Coregistration/spinalcord/warp_T1w*.nii.gz')[0])
    warp_from_PAM50_to_T1w_to_func.append('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-'+subject_name+'/func//5_Coregistration/spinalcord/sub-'+subject_name+'_from-PAM50_to_T1w_to_func_mode-image_xfm.nii.gz')
    warp_from_func_to_T1w_to_PAM50.append('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-'+subject_name+'/func//5_Coregistration/spinalcord/sub-'+subject_name+'_from-func_to_T1w_to_PAM50_mode-image_xfm.nii.gz')
   
    # select data for extraction:
    #data_seed.append(glob.glob(config["coreg_dir"] + 'sub-'+ subject_name +'/'+seed_names[0].split('_')[0]+'/*' + config["coreg_tag"][seed_names[0].split('_')[0]] +'*')[0])
    #data_target.append(glob.glob(config["coreg_dir"] + 'sub-'+ subject_name +'/'+target_name[0].split('_')[0]+'/*' + config["coreg_tag"][target_name[0].split('_')[0]] +'*.gz')[0])
    data_seed.append(glob.glob(config["denoised_dir"] + 'sub-'+ subject_name +'/'+seed_names[0].split('_')[0]+'/*' + config["coreg_tag"][seed_names[0].split('_')[0]] +'*.nii.gz')[0])
    data_target.append(glob.glob(config["denoised_dir"] + 'sub-'+ subject_name +'/'+target_name[0].split('_')[0]+'/*' + config["coreg_tag"][target_name[0].split('_')[0]] +'*.gz')[0])
    
    # Output filename (timeseries)
    ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+target_name[0]+'/test_nativespace/timeseries/sub_' + subject_name + '_mask_' + target_name[0] + '_timeseries')

#>>> Select seed files and ts -------------------------------------
for seed_name in seed_names:
    mask_seed[seed_name]=[];mask_seed_main[seed_name]=[]
    ts_seed_txt[seed_name]=[]
    for subject_name in config['list_subjects']:
        mask_target_inT1w.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/masks/sub_' + subject_name + '_mask_' + target_name[0].split('_')[0] + '_inT1w.nii')
   
        mask_target.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/masks/sub_' + subject_name + '_mask_' + target_name[0].split('_')[0] + '.nii.gz')

        mask_seed_main[seed_name].append(glob.glob(config["main_dir"] + config["seed2vox_dir"] + '/masks/'+ seed_name + '.nii.gz')[0])
        mask_seed[seed_name].append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/masks/sub_' + subject_name + '_mask_' + seed_name.split('_')[-1] + '.nii.gz')
        ts_seed_txt[seed_name].append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/timeseries/sub_' + subject_name + '_mask_' + seed_name.split('_')[-1] + '_timeseries')

#>>> create output directory if needed -------------------------------------
for seed_name in seed_names:
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name + '/test_nativespace/'):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name)
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/timeseries/') # folder to store timeseries extraction
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/brain_fc_maps/') # folder to store maps of FC

 

In [90]:
%cd "/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/func_preparation/codes/code/"
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/func_preparation/codes/code/")
from preprocessings import Preproc
preproc=Preproc(config)
# wrap seed
# create the warping fields by concatenation
for sbj_nb in range(0,len(config['list_subjects'])):
    if not os.path.exists(warp_from_PAM50_to_T1w_to_func[sbj_nb]):
        run_proc('sct_concat_transfo -d {} -w {} {} -o {}'.format(moco_mean[sbj_nb],warp_PAM50_T1w[sbj_nb],warp_T1w_func[sbj_nb],warp_from_PAM50_to_T1w_to_func[sbj_nb]))

# wrap spinal mask into func space    
preproc.coregistration_sc(mask_seed_main[seed_name],moco_mean, mask_seed[seed_name],warp_from_PAM50_to_T1w_to_func,redo=False)

#wrap brain mask to T1w
preproc.normalisation_brain(warp_brain_iy,mask_target_main,mask_target_inT1w,brain_mask=None,n_jobs=1,redo=False)
for sbj_nb in range(0,len(config['list_subjects'])):
    subject_name=config['list_subjects'][sbj_nb]
    if not os.path.exists(mask_target_inT1w[0].split('.')[0]+'.nii.gz'):
        run_proc('fslmaths {} -nan -thr 0.5 -bin '.format(mask_target_inT1w[sbj_nb],mask_target_inT1w[sbj_nb].split('.')[0]+'.nii.gz'))
mask_target_inT1w=[] 
for sbj_nb in range(0,len(config['list_subjects'])):
    subject_name=config['list_subjects'][sbj_nb]
    mask_target_inT1w.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/masks/sub_' + subject_name + '_mask_' + target_name[0].split('_')[0] + '_inT1w.nii.gz')
 
# wrap bmask_target_inT1wrain masl from T1w to func
Preproc.coregistration_flirt_brain(mask_target_inT1w,
                                   moco_brain_mean,
                                   mask_target,
                                   warp_brain_func, binarized=True,redo=False) # put redo=True to rerun that step even if the file already exists
     

/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/func_preparation/codes/code
 
>>>>> Run spinal cord coregistration for 2 participant(s)
Spinal coregistration was already done please put redo=True to redo that step
Spinal coregistration was already done please put redo=True to redo that step
Normalisation into MNI space was already done, set redo=True to run again the coregistration
 
>>>>> Run brain coregistration for 2 participant(s)
convert_xfm -omat /cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-P100/func/5_Coregistration/brain/sub-P100_task-rest_bold_stc_brain_moco_mean_reg_inv.mat -inverse /cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-P100/func/5_Coregistration/brain/sub-P100_task-rest_bold_stc_brain_moco_mean_reg.mat # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/func_preparation/codes/code
flirt -in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//1_first_l

'/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//1_first_level/ica_V1/spinalcord_ica-C6-V/test_nativespace/masks/sub_P105_mask_brain.nii.gz'

### <font color=#4424D6> III. Initialize class based on this </font>

In [91]:
for seed_name in seed_names:
    seed2voxels=Seed2voxels(config,signal,seed_name,target_name) # initialize the function

## <font color=#B2D732> <span style="background-color: #4424D6"> B/ Data extraction </font></span>
### <font color=#4424D6> I. Time series extraction - Target </font>


In [92]:
target_timeseries,target_timeseries_mean,target_timeseries_pc1=seed2voxels.extract_data(img=data_target, mask=mask_target, timeseries_txt=ts_target_txt,
                                                                run="extract",n_jobs=8,smoothing_fwhm=[6,6,6]) # run the analyse for target voxels


### <font color=#4424D6> II. Time series extraction - Seed </font>

In [93]:
seed_timeseries={};seed_timeseries_mean={};seed_timeseries_pc1={}
for seed_name in seed_names:
    seed_timeseries[seed_name],seed_timeseries_mean[seed_name],seed_timeseries_pc1[seed_name]=seed2voxels.extract_data(img=data_seed, mask= mask_seed[seed_name], timeseries_txt=ts_seed_txt[seed_name],
                                                                run="extract",n_jobs=8) # run the analyse for target voxels

## <font color=#B2D732> <span style="background-color: #4424D6"> C/ Correlation analysis

In [94]:
output_img={};

for seed_name in seed_names:
    output_img[seed_name]=[]
    output_img[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/'+target_name[0].split('_')[0]+ '_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_seed_' + seed_name.split('_')[-1] + '_s'

    seed2voxels.correlation_maps(seed_timeseries_mean[seed_name],target_timeseries,mask_files=mask_target,output_img=output_img[seed_name],Fisher=True,partial=False,save_maps=True,redo=False,n_jobs=8)


ValueError: Field of view of image #1 is different from reference FOV.
Reference affine:
array([[ -1.60000002,   0.        ,  -0.        ,  95.99998474],
       [  0.        ,   1.58442891,   0.61236167, -92.11682129],
       [  0.        ,  -0.22267698,   4.35717964,  43.74253464],
       [  0.        ,   0.        ,   0.        ,   1.        ]])
Image affine:
array([[-1.60000002e+00,  0.00000000e+00, -0.00000000e+00,
         9.89999847e+01],
       [-0.00000000e+00,  1.59701574e+00, -2.68613577e-01,
        -8.22239227e+01],
       [ 0.00000000e+00,  9.76776704e-02,  4.39179325e+00,
         1.02370148e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])
Reference shape:
(120, 120, 35)
Image shape:
(120, 120, 35, 1)


In [120]:
corr_maps=[];corr_maps_inT1w=[]; anat_brain=[]
for sbj_nb in range(0,len(config['list_subjects'])):
    subject_name=config['list_subjects'][sbj_nb]
    corr_maps.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/'+target_name[0].split('_')[0]+ '_fc_maps/sub-' + subject_name + '_SPM.nii')
    corr_maps_inT1w.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/'+target_name[0].split('_')[0]+ '_fc_maps/sub-' + subject_name + '_inT1w.nii')
    anat_brain.append(glob.glob('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/'+ 'sub-' + subject_name + '/anat/brain/segmentation/*brain_masked.nii.gz')[0])

preproc.coregistration_SPM_brain(anat_brain,
                         corr_maps,[""],
                         corr_maps_inT1w,[""],
                         n_jobs=1,redo=True)

['']
Unzip data for SPM
Unzip done for: sub-P100_T1w_brain_masked.nii.gz
Unzip done for: sub-P105_T1w_brain_masked.nii.gz
Coregistration into anat space done


In [123]:
#wrap brain mask to T1w
warp_brain_y=[];corr_maps_inMNI=[]
for sbj_nb in range(0,len(config['list_subjects'])):
    subject_name=config['list_subjects'][sbj_nb]
    warp_brain_y.append(glob.glob('/cerebro/cerebro1/dataset/bmpd/derivatives/spinalcord_processing/sub-'+subject_name+'/anat/brain/segmentation/mri/y*')[0])
    corr_maps_inMNI.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/ica_V1/'+seed_name+'/test_nativespace/'+target_name[0].split('_')[0]+ '_fc_maps/sub-' + subject_name + '_inMNI.nii')
preproc.normalisation_brain(warp_brain_y,corr_maps_inT1w,
                            corr_maps_inMNI,brain_mask=None,n_jobs=1,redo=False)



Normalisation to MNI space Done
output: sub-P100_inMNI.nii
Normalisation to MNI space Done
output: sub-P105_inMNI.nii
Normalisation into MNI space done


## <font color=#B2D732> <span style="background-color: #4424D6"> D/ Statistics
#### <font color=#4424D6> One sample t-test

In [ ]:
seed_name=seed_names[0]
ouput_dir='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/2_second_level/' + target_name[0] + '/spinalcord_GM/'+seed_name +'/'
if not os.path.exists('ouput_dir'):
    #os.mkdir(ouput_dir)
    run_proc('randomise -i {} -o {} -1 -v 5 -n 500  -T'.format(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' + seed_name + '/' + target_name[0].split('_')[0]+ '_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_seed_' + seed_name.split('_')[-1] + '_s_zcorr.nii',
                                                           ouput_dir + seed_name ,
                                                           mask_target))

#### <font color=#4424D6> Two sample t-test

In [22]:
import shutil
#1. select the analyses to compared
#'spinalcord_GM-C4C5L','spinalcord_GM-C4C5R',
#'spinalcord_GM-C6C8L','spinalcord_GM-C6C8R'
        
seed1='spinalcord_GM-C6C8R'
seed2='spinalcord_GM-C6C8L'
output_dir=config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/'+seed1+'-'+seed2.split('-')[-1]
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
#2. merge the images from the 2 contrasts in one 4D image
    run_proc('fslmerge -t {} {} {}'.format(output_dir + '/'+seed1+'-'+seed2.split('-')[-1]+'_images.nii', 
                                    config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' +seed1 + '/' + target_name[0].split('_')[0]+ '_fc_maps/21subjects_seed_'+seed1.split('_')[-1]+'_s_zcorr',
                                    config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' +seed2 + '/' + target_name[0].split('_')[0]+ '_fc_maps/21subjects_seed_'+seed2.split('_')[-1]+'_s_zcorr'))

#3. Run randomize
    run_proc('randomise -i {} -o {} -d {} -t {} -e {} -m {} -v 5 -n 500  -T'.format(output_dir + '/'+seed1+'-'+seed2.split('-')[-1]+'_images.nii',
                                                                                    output_dir + '/'+seed1+'-'+seed2.split('-')[-1],
                                                                                    config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/fsl_designs/2paired-t-test/design_group.mat',
                                                                                    config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/fsl_designs/2paired-t-test/design_group.con',
                                                                                    config['main_dir'] + config['seed2vox_dir'] + '/2_second_level/fsl_designs/2paired-t-test/design_group.grp',
                                                                                mask_target))

fslmerge -t /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//2_second_level/spinalcord_GM-C6C8R-C6C8L/spinalcord_GM-C6C8R-C6C8L_images.nii /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//1_first_level/spinalcord_GM-C6C8R/brain_fc_maps/21subjects_seed_GM-C6C8R_s_zcorr /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//1_first_level/spinalcord_GM-C6C8L/brain_fc_maps/21subjects_seed_GM-C6C8L_s_zcorr # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/notebook


RuntimeError: Image Exception : #63 :: No image files match: /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//1_first_level/spinalcord_GM-C6C8R/brain_fc_maps/21subjects_seed_GM-C6C8R_s_zcorr
terminate called after throwing an instance of 'std::runtime_error'
what():  No image files match: /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//seed_to_voxels//1_first_level/spinalcord_GM-C6C8R/brain_fc_maps/21subjects_seed_GM-C6C8R_s_zcorr

In [98]:
input='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_C1C8-L.nii.gz'
mask='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-cord.nii.gz'
output='/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-L.nii.gz'
run_proc('fslmaths {} -mas {} {}'.format(input,mask,output))

fslmaths /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_C1C8-L.nii.gz -mas /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-cord.nii.gz /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/seed_to_voxels/masks/spinalcord_GM-L.nii.gz # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/notebook


(0, '')